In [ ]:
# 高德API返回经纬度
import requests
import json
import pandas as pd

def request_url_get(url):
    try:
        r = requests.get(url=url, timeout=30)
        if r.status_code == 200:
            return r.text
        return None
    except RequestException:
        print('请求url返回错误异常')
        return None

def parse_json(content_json):
    result_json = json.loads(content_json)
    return result_json

def request_api(url):
    result = request_url_get(url)
    result_json = parse_json(result)
    return result_json

key = "c40f7633aa161704028ba1c095f2f1c3"
batch = "false"

df = pd.read_excel(".\data\stores.xlsx")
df_data = pd.DataFrame(columns=["城市", "名称", "地址", "经度(api)", "纬度(api)"])
len_df = len(df)
for i in range(len_df):
    city = df.iloc[i, 1]
    name = df.iloc[i, 2]
    address = df.iloc[i, 3]
    url = f'https://restapi.amap.com/v3/geocode/geo?key={key}&address={address}&city={city}&batch={batch}'
    ret = request_api(url)
    pos = ret["geocodes"][0]["location"].split(',')

    df_data = df_data.append([{
        "城市": city,
        "名称": name,
        "地址": address,
        "经度(api)": pos[0],
        "纬度(api)": pos[1]
        }], ignore_index=True)

# df_data.to_excel(".\data\stores_api.xlsx")

In [ ]:
# 计算门店间距离
from math import pi, cos, sin, acos, asin, radians, pow, sqrt
import pandas as pd

# def distance(x_lon, x_lat, y_lon, y_lat):
#     R = 6371
#     x_lon = pi / 180 * x_lon
#     x_lat = pi / 180 * x_lat
#     y_lon = pi / 180 * y_lon
#     y_lat = pi / 180 * y_lat
#     return acos(sin(x_lat) * sin(y_lat) + cos(x_lat) * cos(y_lat) * cos(y_lon - x_lon)) * R

def haversin(theta):
    return pow(sin(theta/2), 2)

def distance(x_lng, x_lat, y_lng, y_lat):
    EARTH_RADIUS = 6371
    x_lng = radians(x_lng)
    x_lat = radians(x_lat)
    y_lng = radians(y_lng)
    y_lat = radians(y_lat)

    d_lng = abs(y_lng - x_lng)
    d_lat = abs(y_lat - x_lat)

    h = haversin(d_lat) + cos(x_lat) * cos(y_lat) * haversin(d_lng)

    return 2 * EARTH_RADIUS * asin(sqrt(h))

def filter(x_lng, x_lat, y_lng, y_lat, x_name, y_name, x_address, y_address, threshold=5):
    dist = distance(x_lng, x_lat, y_lng, y_lat)
    if dist <= threshold:
        return True, [{
            "门店A": x_name,
            "门店A地址": x_address,
            "门店B": y_name,
            "门店B地址": y_address,
            "距离(公里)": dist,
        }]
    else:
        return False, []

df_data = pd.read_excel(".\data\stores_api.xlsx")
df_dist = pd.DataFrame(columns=["门店A", "门店A地址", "门店B", "门店B地址", "距离(公里)"])
length = len(df_data)
threshold = 5

for i in range(length):
    for j in range(i+1, length):
        dist = distance(df_data.iloc[i, 4], df_data.iloc[i, 5], df_data.iloc[j, 4], df_data.iloc[j, 5])
        if dist <= threshold:
            df_dist = df_dist.append([{
                "门店A": df_data.iloc[i, 2],
                "门店A地址": df_data.iloc[i, 3],
                "门店B": df_data.iloc[j, 2],
                "门店B地址": df_data.iloc[j, 3],
                "距离(公里)": dist,
            }], ignore_index=True)

df_dist.sort_values("距离(公里)", inplace=True)

df_dist.to_excel(".\data\distance.xlsx")

In [ ]:
# 单条个别处理
# 高德API返回经纬度
import requests
import json
import pandas as pd

def request_url_get(url):
    try:
        r = requests.get(url=url, timeout=30)
        if r.status_code == 200:
            return r.text
        return None
    except RequestException:
        print('请求url返回错误异常')
        return None

def parse_json(content_json):
    result_json = json.loads(content_json)
    return result_json

def request_api(url):
    result = request_url_get(url)
    result_json = parse_json(result)
    return result_json

key = "c40f7633aa161704028ba1c095f2f1c3"
batch = "true"

city = "石家庄市"

# name = "石家庄工农路店"
# address = "河北省石家庄市桥西区工农路华域一层商铺12号"

address = "石家庄市桥西区工农路623号|河北省石家庄市桥西区工农路12号华域一层商铺"
url = f'https://restapi.amap.com/v3/geocode/geo?key={key}&address={address}&city={city}&batch={batch}'
ret = request_api(url)
print(ret)
# pos = ret["geocodes"][0]["location"].split(',')

# r = {
#     "城市": city,
#     "名称": name,
#     "地址": address,
#     "经度(api)": pos[0],
#     "纬度(api)": pos[1]
# }
# print(r)
